In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from ipynb.fs.full.utility_functions import gen_data_dict, get_data, error_count, write_to_csv, opart, SquaredHingeLoss

np.set_printoptions(precision=3)
np.random.seed(123)
torch.manual_seed(123)

In [2]:
# data
data = pd.read_csv('1.genome_learning_data/seq_features.csv')['count'].to_numpy()
data = np.log(np.log(data)).reshape(-1,1)
data = torch.FloatTensor(data)

target_df_1 = pd.read_csv('1.genome_learning_data/target_lambda_fold1.csv')
target_df_2 = pd.read_csv('1.genome_learning_data/target_lambda_fold2.csv')

targets_low_1  = target_df_1.iloc[:, 1:2].to_numpy()
targets_high_1 = target_df_1.iloc[:, 2:3].to_numpy()
targets_low_2  = target_df_2.iloc[:, 1:2].to_numpy()
targets_high_2 = target_df_2.iloc[:, 2:3].to_numpy()

targets_low_1  = torch.FloatTensor(targets_low_1)
targets_high_1 = torch.FloatTensor(targets_high_1)
targets_low_2  = torch.FloatTensor(targets_low_2)
targets_high_2 = torch.FloatTensor(targets_high_2)

y1 = torch.cat((targets_low_1, targets_high_1), dim=1)
y2 = torch.cat((targets_low_2, targets_high_2), dim=1)

In [3]:
# Define the linear model
class LinearModel(nn.Module):
    def __init__(self, input_size):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.fc1(x)

In [4]:
# Instantiate the model, define custom loss function, and optimizer
model1 = LinearModel(input_size = 1)
model2 = LinearModel(input_size = 1)

squared_hinge_loss = SquaredHingeLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)

# Training loop
min_loss_1_test = float('inf')
min_loss_2_test = float('inf')
for epoch in range(1000):
    # Forward pass
    outputs1 = model1(data)
    outputs2 = model2(data)
    
    # Compute the custom loss
    loss_1 = squared_hinge_loss(outputs1, y1)
    loss_2 = squared_hinge_loss(outputs2, y2)

    loss_1_test = squared_hinge_loss(outputs1, y2)
    loss_2_test = squared_hinge_loss(outputs2, y1)
    
    # Backward pass and optimization
    optimizer1.zero_grad()
    loss_1.backward()
    optimizer1.step()

    optimizer2.zero_grad()
    loss_2.backward()
    optimizer2.step()

    # save models
    if loss_1_test < min_loss_1_test:
        min_loss_1_test = loss_1_test
        torch.save(model1.state_dict(), '1.genome_saved_models/model1_linear_best.pth')
    
    if loss_2_test < min_loss_2_test:
        min_loss_2_test = loss_2_test
        torch.save(model2.state_dict(), '1.genome_saved_models/model2_linear_best.pth')
    
    # Print the loss every 100 epochs
    if (epoch) % 50 == 0:
        print(f'Epoch {epoch:5d}, Loss_1: {loss_1.item():8.4f}, Loss_1_test: {loss_1_test.item():8.4f}, Loss_2: {loss_2.item():8.4f}, Loss_2_test: {loss_2_test.item():8.4f}')

Epoch     0, Loss_1:   2.3556, Loss_1_test:   2.1364, Loss_2:   1.8813, Loss_2_test:   1.9977
Epoch    50, Loss_1:   2.0690, Loss_1_test:   1.9265, Loss_2:   1.7379, Loss_2_test:   1.7776
Epoch   100, Loss_1:   1.8537, Loss_1_test:   1.7819, Loss_2:   1.6439, Loss_2_test:   1.6182
Epoch   150, Loss_1:   1.6919, Loss_1_test:   1.6846, Loss_2:   1.5778, Loss_2_test:   1.4968
Epoch   200, Loss_1:   1.5613, Loss_1_test:   1.6117, Loss_2:   1.5315, Loss_2_test:   1.4032
Epoch   250, Loss_1:   1.4506, Loss_1_test:   1.5526, Loss_2:   1.5162, Loss_2_test:   1.3562
Epoch   300, Loss_1:   1.3610, Loss_1_test:   1.5093, Loss_2:   1.5126, Loss_2_test:   1.3360
Epoch   350, Loss_1:   1.3113, Loss_1_test:   1.5017, Loss_2:   1.5115, Loss_2_test:   1.3251
Epoch   400, Loss_1:   1.2840, Loss_1_test:   1.5101, Loss_2:   1.5108, Loss_2_test:   1.3175
Epoch   450, Loss_1:   1.2627, Loss_1_test:   1.5215, Loss_2:   1.5105, Loss_2_test:   1.3117
Epoch   500, Loss_1:   1.2451, Loss_1_test:   1.5346, Loss_2

In [5]:
# torch.save(model1.state_dict(), '1.genome_saved_models/model1_linear_best.pth')
# torch.save(model2.state_dict(), '1.genome_saved_models/model2_linear_best.pth')

In [6]:
# Load model1
model1 = LinearModel(input_size=1)
model1.load_state_dict(torch.load('1.genome_saved_models/model1_linear_best.pth'))
model1.eval()  # Set the model to evaluation mode

# Load model2
model2 = LinearModel(input_size=1)
model2.load_state_dict(torch.load('1.genome_saved_models/model2_linear_best.pth'))
model2.eval()  # Set the model to evaluation mode

LinearModel(
  (fc1): Linear(in_features=1, out_features=1, bias=True)
)

In [7]:
with torch.no_grad():
    ldas1 = model1(data).numpy().reshape(-1)
    ldas2 = model2(data).numpy().reshape(-1)

In [8]:
seqs   = gen_data_dict('sequence_label_data/genome/signals.gz')
labels = gen_data_dict('sequence_label_data/genome/labels.gz')

header = ['sequenceID', 'fold_1_total_labels', 'fold_2_total_labels', 'fold_1_fp_errs', 'fold_1_fn_errs', 'fold_1_tp', 'fold_1_tn', 'fold_2_fp_errs', 'fold_2_fn_errs', 'fold_2_tp', 'fold_2_tn',]

for i in range(len(seqs)):
    # generate data
    sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2 = get_data(i, seqs=seqs, labels=labels)
    sequence_length = len(sequence)-1

    # get total labels
    fold1_total_labels = len(neg_start_1) + len(pos_start_1)
    fold2_total_labels = len(neg_start_2) + len(pos_start_2)

    # run each lambda and record it into csv file
    row = [seqs[i][0], fold1_total_labels, fold2_total_labels]

    chpnt_fold1 = opart(10**ldas2[i], sequence)
    chpnt_fold2 = opart(10**ldas1[i], sequence)

    err_1 = error_count(chpnt_fold1, neg_start_1, neg_end_1, pos_start_1, pos_end_1)
    err_2 = error_count(chpnt_fold2, neg_start_2, neg_end_2, pos_start_2, pos_end_2)
    
    row.append(err_1[0])
    row.append(err_1[1])
    row.append(err_1[2])
    row.append(err_1[3])
    row.append(err_2[0])
    row.append(err_2[1])
    row.append(err_2[2])
    row.append(err_2[3])

    write_to_csv('1.genome_learning_output/linear_test.csv', header, row)